# Basic data access 
This notebook showcases helper functions from `climakitae` that enable you to access and export the AE catalog data, while also allowing you to perform spatial subsetting and view the data options in an easy-to-use fashion. These functions could be easily implemented in a python script. 

In [1]:
import climakitae as ck 
from climakitae.core.data_interface import (
    get_data_options, 
    get_subsetting_options, 
    get_data
)

## High-level details 
The AE data catalog has many different types of data. Our helper library `climakitae` attempts to make accessing and retrieveing this data intuitive, as well as simplify climate and statistical analysis with the data down the line, by performing some data transformations as the data is retrieved.<br><br> To retrieve the data, you'll need to make some selections as to your climate variable, data resolution, location settings, and many other options. There are also several high-level options you'll need to set when selecting your data, detailed below: 

### Data type: Gridded or Stations
**Gridded**: Gridded (i.e. raster) climate data at various spatial resolutions.<br><br>
**Stations**: Gridded (i.e. raster) climate data at unique grid cell(s) corresponding to the central coordinates of the selected weather station(s). 
- This data is bias-corrected (i.e localized) to the exact location of the weather station using the historical in-situ data from the weather station(s). 
- This data is currently only available for dynamically downscaled air temperature data. 

### Scientific approach: Time or Warming Level
**Time**: Retrieve the data using a traditional time-based approach that allows you to select historical data, future projections, or both, along with a time-slice of interest. 
- “Historical Climate” includes data from 1980-2014 simulated from the same GCMs used to produce the Shared Socioeconomic Pathways (SSPs). It will be automatically appended to a SSP time series when both are selected. Because this historical data is obtained through simulations, it represents average weather during the historical period and is not meant to capture historical timeseries as they occurred.
- “Historical Reconstruction” provides a reference downscaled [reanalysis](https://www.ecmwf.int/en/about/media-centre/focus/2020/fact-sheet-reanalysis) dataset based on atmospheric models fit to satellite and station observations, and as a result will reflect observed historical time-evolution of the weather.
- Future projections are available for [greenhouse gas emission scenario (Shared Socioeconomic Pathway, or SSP)](https://climatescenarios.org/primer/socioeconomic-development) SSP 3-7.0 through 2100 with the dynamically-downscaled General Circulation Models (GCMs).
     - One GCM was additionally downscaled for two additional SSPs (SSP 5-8.5 and SSP 2-4.5)<br>

**Warming Level**: Retrieve the data by future global warming levels, which will automatically retrieve all available model data for the historical+future period and then calculate the time window around which each simulation reaches the selected warming level.  
- Because warming levels are defined based on amount of global mean temperature change, they can be used to compare possible outcomes across multiple scenarios or model simulations.
- This approach includes all simulations that reach a specified amount of warming regardless of when they reach that level of warming, rather than the time-based appraoch, which will preliminarily subset a portion of simulations that follow a given SSP trajectory.
    
### Downscaling method: Dynamical, Statistical, or both
**Dynamical**:[Dynamically downscaled](https://dept.atmos.ucla.edu/alexhall/downscaling-cmip6) WRF data, produced at hourly intervals. If you select 'daily' or 'monthly' for 'Timescale', you will receive an average of the hourly data. The spatial resolution options, on the other hand, are each the output of a different simulation, nesting to higher resolution over smaller areas.<br><br>
**Statistical**: [Hybrid-statistically downscaled](https://loca.ucsd.edu) LOCA2-Hybrid data, available at daily and monthly timescales. Multiple LOCA2-Hybrid simulations are available (100+) at a fine spatial resolution of 3km.

## See the options in our data catalog in a table
This function returns a pandas DataFrame (i.e. a table) of our data options. You can also use the library `climakitaegui` to visualize these options in an interactive panel. See the notebook `interactive_data_access_and_viz.ipynb` to explore that approach. 

In [2]:
options_df = get_data_options()
options_df.head(10)


variable  \
downscaling_method scenario           timescale                                         
Statistical        Historical Climate daily                 Maximum relative humidity   
                                      daily                 Minimum relative humidity   
                                      daily                   Specific humidity at 2m   
                                      daily                     Precipitation (total)   
                                      daily             Shortwave flux at the surface   
                                      daily             Maximum air temperature at 2m   
                                      daily             Minimum air temperature at 2m   
                                      daily        West-East component of Wind at 10m   
                                      daily      North-South component of Wind at 10m   
                                      daily                         Wind speed at 10m   

                                                resolution  
downscaling_method scenario           timescale             
Statistical        Historical Climate daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
                                      daily           3 km

In [3]:
options_df.to_csv('data_options.csv')

## See the data options for a particular subset of inputs
The `get_data_options` function enables you to input a number of different function arguments, corresponding to the columns in the table above, to subset the table. Inputting no arguments, like we did above, will return the entire range of options.<br><br>First, lets print the function documentation to see the inputs and outputs of the function. If an argument (or "parameter", as listed in the documentation) is listed as "optional", that means you don't have to input anything for that argument. In the case of this function, none of the function arguments are required, so you can simply call the function. 

In [4]:
print(get_data_options.__doc__)

Get data options, in the same format as the Select GUI, given a set of possible inputs.
    Allows the user to access the data using the same language as the GUI, bypassing the sometimes unintuitive naming in the catalog.
    If no function inputs are provided, the function returns the entire AE catalog that is available via the Select GUI

    Parameters
    ----------
    variable : str, optional
        Default to None
    downscaling_method : str, optional
        Default to None
    resolution : str, optional
        Default to None
    timescale : str, optional
        Default to None
    scenario : str or list, optional
        Default to None
    tidy : boolean, optional
        Format the pandas dataframe? This creates a DataFrame with a MultiIndex that makes it easier to parse the options.
        Default to True
    enable_hidden_vars : boolean, optional
        Return all variables, including the ones in which "show" is set to False?
        Default to False

    Returns
  

If you call the function with **no inputs**, it will simply return the entire catalog! But, let's say you want to see all the data options for statistically downscaled data at 3 km resolution. You'll want to provide inputs for the `downscaling_method` and `resolution` arguments. 

In [5]:
get_data_options(
    downscaling_method = "Statistical", 
    resolution = "3 km"
)

variable  \
downscaling_method scenario           timescale                                         
Statistical        Historical Climate daily                 Maximum relative humidity   
                                      daily                 Minimum relative humidity   
                                      daily                   Specific humidity at 2m   
                                      daily                     Precipitation (total)   
                                      daily             Shortwave flux at the surface   
...                                                                               ...   
                   SSP 5-8.5          monthly           Maximum air temperature at 2m   
                                      monthly           Minimum air temperature at 2m   
                                      monthly      West-East component of Wind at 10m   
                                      monthly    North-South component of Wind at 10m   
                                      monthly                       Wind speed at 10m   

                                                resolution  
downscaling_method scenario           timescale             
Statistical        Historical Climate daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
...                                                    ...  
                   SSP 5-8.5          monthly         3 km  
                                      monthly         3 km  
                                      monthly         3 km  
                                      monthly         3 km  
                                      monthly         3 km  

[80 rows x 2 columns]

Perhaps you want to see all the data options for daily precipitation. We have several precipitation options in the catalog. You don't need to know the name of these variables; simply use "precipitation" as your input to the function for the `variable` argument.<br><br>The function prefers that your inputs match an actual option in the catalog-- with exact capitalizations and no misspelling-- and will print a warning if your input is not a direct match ("precipitation" is not an option, but "Precipitation (total)" is). The function will then try to make a guess as to what you actually meant. 

In [7]:
get_data_options(
    variable = "monthly", 
    timescale = "daily"
)

Input variable='monthly' is not a valid option.
Valid options: 
- Air Temperature at 2m
- Dew point temperature
- Effective Temperature
- Evaporation
- Fosberg fire weather index
- Ground heat flux
- Ice water path
- Instantaneous downwelling clear sky longwave flux at bottom
- Instantaneous downwelling clear sky shortwave flux at bottom
- Instantaneous downwelling longwave flux at bottom
- Instantaneous downwelling shortwave flux at bottom
- Instantaneous upwelling clear sky longwave flux at bottom
- Instantaneous upwelling clear sky shortwave flux at bottom
- Instantaneous upwelling longwave flux at bottom
- Instantaneous upwelling shortwave flux at bottom
- Latent heat flux at the surface
- Liquid water path
- Longwave flux at the surface
- Maximum air temperature at 2m
- Maximum precipitation
- Maximum relative humidity
- Maximum wind speed at 10m
- Mean wind speed at 10m
- Minimum air temperature at 2m
- Minimum relative humidity
- NOAA Heat Index
- North-South component of Wind a

ValueError: Bad input

The function can also return a simple pandas DataFrame without the complex MultiIndex. Just set `tidy = False`.

In [8]:
get_data_options(
    variable = "precipitation", 
    timescale = "daily", 
    tidy = False
) 

Input variable='precipitation' is not a valid option.
Closest options: 
- Maximum precipitation
- Precipitation (cumulus portion only)
- Precipitation (grid-scale portion only)
- Precipitation (total)
Outputting data for variable='Maximum precipitation'



,variable,downscaling_method,resolution,timescale,scenario
0,Maximum precipitation,Dynamical,45 km,daily,Historical Climate
1,Maximum precipitation,Dynamical,9 km,daily,Historical Climate
2,Maximum precipitation,Dynamical,3 km,daily,Historical Climate
3,Maximum precipitation,Dynamical,45 km,daily,SSP 2-4.5
4,Maximum precipitation,Dynamical,9 km,daily,SSP 2-4.5
5,Maximum precipitation,Dynamical,45 km,daily,SSP 3-7.0
6,Maximum precipitation,Dynamical,9 km,daily,SSP 3-7.0
7,Maximum precipitation,Dynamical,3 km,daily,SSP 3-7.0
8,Maximum precipitation,Dynamical,45 km,daily,SSP 5-8.5
9,Maximum precipitation,Dynamical,9 km,daily,SSP 5-8.5


## See all the geometry options for spatially subsetting the data during retrieval
These options will match those in our AE selections GUI. This will enable you to retrieve a subset for a specific region.

In [9]:
subset_options_df = get_subsetting_options()
subset_options_df.head(10)

geometry
area_subset cached_area                                                   
states      ID           POLYGON ((-117.24269 44.39655, -117.23485 44.3...
            WA           MULTIPOLYGON (((-122.57041 48.53786, -122.5686...
            NM           POLYGON ((-109.05018 31.48001, -109.04985 31.4...
            CA           MULTIPOLYGON (((-118.60443 33.47856, -118.5988...
            CO           POLYGON ((-109.06026 38.59933, -109.05955 38.7...
            UT           POLYGON ((-114.05297 37.59279, -114.05248 37.6...
            WY           POLYGON ((-111.05457 45.00096, -111.04508 45.0...
            NV           POLYGON ((-120.00576 39.22867, -120.00561 39.2...
            MT           POLYGON ((-116.04915 48.50205, -116.04914 48.5...
            AZ           POLYGON ((-114.81631 32.50804, -114.81433 32.5...

This shows a lot of options! Say you're only interested in California counties. Simply set the argument `area_subset` to "CA counties" to see the all options for counties. The function documentation shows the other options, which also match the values in the column "area_subset" in the table above. 

In [10]:
print(get_subsetting_options.__doc__)

Get all geometry options for spatial subsetting.
    Options match those in selections GUI

    Parameters
    ----------
    area_subset : str
        One of "all", "states", "CA counties", "CA Electricity Demand Forecast Zones", "CA watersheds", "CA Electric Balancing Authority Areas", "CA Electric Load Serving Entities (IOU & POU)", "Stations"
        Defaults to "all", which shows all the geometry options with area_subset as a multiindex

    Returns
    -------
    geom_df : pd.DataFrame
        Geometry options
        Shows only options for one area_subset if input is provided that is not "all"
        i.e. if area_subset = "states", only the options for states will be returned

    


In [11]:
get_subsetting_options(area_subset = "CA counties")

,geometry
cached_area,
Alameda County,"POLYGON ((-122.37312 37.88388, -122.37378 37.8..."
Alpine County,"POLYGON ((-120.07333 38.70109, -120.07332 38.7..."
Amador County,"POLYGON ((-121.02771 38.50011, -121.02771 38.5..."
Butte County,"POLYGON ((-122.06943 39.84053, -122.06886 39.8..."
Calaveras County,"POLYGON ((-120.6318 38.34603, -120.6318 38.345..."
Colusa County,"POLYGON ((-121.91512 38.92535, -121.91491 38.9..."
Contra Costa County,"POLYGON ((-121.69732 37.78244, -121.69084 37.7..."
Del Norte County,"POLYGON ((-124.31611 41.72839, -124.3137 41.72..."
El Dorado County,"POLYGON ((-120.18443 39.03101, -120.18838 39.0..."


You can see all the options for subsetting, and their corresponding geometries, but you don't actually need to use the geometries for subsetting if you use climakitae's data retrieval function-- `get_catalog_data` -- explained in the next section. 

## Retrieve data using the get_data() function
You can easily retrieve data from the Analytics Engine data catalog using climakitae's ```get_data``` function, described below. Additional details for each of the function arguments can be viewed in function docstrings in the next code cell. 

### Required inputs 
This function requires you to input values for the following arguments: 
- variable (required)
- resolution (required)
- timescale (required)

### Location subsetting 
The options for location subsetting can be found using the `get_data_options()` function, as described in the beginning of this notebook. You can also opt to perform an area average by setting `area_average = "Yes"`. The `get_data()` function will default to returning the entire spatial domain, with no area averaging performed. 
- area_subset (optional) 
- cached_area (optional) 
- area_average (optional)

### Additional options
Further modify the data returned using the following arguments.
- downscaling method (optional)
- approach (optional) 
- scenario (optional)
- units (optional)
- time_slice (optional)
- warming_level (optional)
- warming_level_window (optional)
- warming_level_months (optional)

In [12]:
# See additional details about the function arguments by printing the docstring
print(get_data.__doc__)

Retrieve formatted data from the Analytics Engine data catalog using a simple function.
    Contrasts with DataParameters().retrieve(), which retrieves data from the user inputs in climakitaegui's selections GUI.

    Parameters
    ----------
    variable : str
        String name of climate variable
    resolution : str, one of ["3 km", "9 km", "45 km"]
        Resolution of data in kilometers
    timescale : str, one of ["hourly", "daily", "monthly"]
        Temporal frequency of dataset
    downscaling_method : str, one of ["Dynamical", "Statistical", "Dynamical+Statistical"], optional
        Downscaling method of the data:
        WRF ("Dynamical"), LOCA2 ("Statistical"), or both "Dynamical+Statistical"
        Default to "Dynamical"
    data_type : str, one of ["Gridded", "Stations"], optional
        Whether to choose gridded data or weather station data
        Default to "Gridded"
    approach : one of ["Time", "Warming Level"], optional
        Default to "Time"
    scenario

### Example 1: Time-based approach
Retrieve gridded data using a time-based approach. ```approach``` is an optional function argument, but the default is to use a time-based approach, so you don't actually need to set this argument. 

#### Example 1a
First, let's retrieve 3 kilometer resolution statistically downscaled historical data at a monthly timestep. 

In [13]:
get_data(
    variable = "Precipitation (total)", 
    downscaling_method = "Statistical", 
    resolution = "3 km", 
    timescale = "monthly", 
    scenario = "Historical Climate"
    # approach = "Time" # Optional because "Time" is the function default 
)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!! Returned data array is huge. Operations could take 10x to infinity longer than 1GB of data !!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



<xarray.DataArray np.str_('Precipitation (total)') (scenario: 1,
                                                    simulation: 70, time: 780,
                                                    lat: 495, lon: 559)> Size: 121GB
dask.array<broadcast_to, shape=(1, 70, 780, 495, 559), dtype=float64, chunksize=(1, 1, 308, 310, 351), chunktype=numpy.ndarray>
Coordinates:
  * lat          (lat) float32 2kB 29.58 29.61 29.64 29.67 ... 44.95 44.98 45.02
  * lon          (lon) float32 2kB -128.4 -128.4 -128.4 ... -111.0 -111.0 -111.0
  * time         (time) datetime64[ns] 6kB 1950-01-01 1950-02-01 ... 2014-12-01
  * simulation   (simulation) <U30 8kB 'LOCA2_CNRM-ESM2-1_r1i1p1f2' ... 'LOCA...
  * scenario     (scenario) <U18 72B 'Historical Climate'
    spatial_ref  int64 8B 0
Attributes:
    variable_id:           pr
    extended_description:  Total precipitation. Computed by summing total gri...
    units:                 kg m-2 s-1
    data_type:             Gridded
    resolution:            3 km
    frequency:             monthly
    location_subset:       ['entire domain']
    approach:              Time
    downscaling_method:    Statistical
    institution:           UCSD

#### Example 1b
Now say you're only interested in this data for San Bernadino County, and you want to compute an area average over the entire county. 

In [14]:
get_data(
    variable = "Precipitation (total)", 
    downscaling_method = "Statistical", 
    resolution = "3 km", 
    timescale = "monthly", 
    scenario = "Historical Climate",
    
    # Modify location settings
    cached_area = "San Bernardino County", 
    area_average = "Yes"
)

<xarray.DataArray np.str_('Precipitation (total)') (scenario: 1,
                                                    simulation: 70, time: 780)> Size: 437kB
dask.array<broadcast_to, shape=(1, 70, 780), dtype=float64, chunksize=(1, 1, 308), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 6kB 1950-01-01 1950-02-01 ... 2014-12-01
  * simulation   (simulation) <U30 8kB 'LOCA2_GFDL-ESM4_r1i1p1f1' ... 'LOCA2_...
  * scenario     (scenario) <U18 72B 'Historical Climate'
    spatial_ref  int64 8B 0
Attributes:
    variable_id:           pr
    extended_description:  Total precipitation. Computed by summing total gri...
    units:                 kg m-2 s-1
    data_type:             Gridded
    resolution:            3 km
    frequency:             monthly
    location_subset:       ['San Bernardino County']
    approach:              Time
    downscaling_method:    Statistical
    institution:           UCSD

#### Example 1c 
Perhaps next you want to get dynamically downscaled (i.e. WRF) precipitation data instead. First, you might want to check what options you have for scenario, timescale, and resolution using the ```get_data_options()``` function. 

In [ ]:
get_data_options(
    variable = "Precipitation (total)", 
    downscaling_method = "Dynamical"
) 

Next, let's retrieve both the future and historical dynamically downscaled data. "Historical Climate" is the correct historical data option here; "Historical Reconstruction" data is from ERA5 (a climate reanalysis product, rather than a climate model), and cannot be retrieved with future data in the same function call. <br><br>You can set the ```scenario``` argument to retrieve the shared socioeconomic pathway data (future projections) appended to the historical data. You can also set your desired time period using the ```time_slice``` argument. 

In [ ]:
get_data(
    variable = "Precipitation (total)", 
    downscaling_method = "Dynamical", 
    resolution = "45 km", 
    timescale = "monthly", 
    cached_area = "San Bernardino County", 
    
    # Modify time-based settings 
    time_slice = (2000,2050),
    scenario = [
        "Historical Climate", 
        "SSP 3-7.0", 
        "SSP 2-4.5",
        "SSP 5-8.5"
    ]
) 

### Example 2: Warming levels approach 
By default, the function uses a time-based approach. To use a warming levels approach, set the argument ```approach = "Warming Level"```. 

#### Example 2a
Retrieve the same data as example 1c, using a warming levels approach instead of a time-based approach. Note that the ```scenario``` and ```time_slice``` arguments are invalid for a warming levels approach; if provided, they will be ignored by the function. 

In [ ]:
get_data(
    variable = "Precipitation (total)", 
    downscaling_method = "Dynamical", 
    resolution = "45 km", 
    timescale = "monthly", 
    cached_area = "San Bernardino County", 
    
    # Modify your approach 
    approach = "Warming Level",
)

#### Example 2b
The ```get_data()``` function uses a default warming levels window of +/- 15 years, resulting in a 30 year period. Lets modify that by setting ```warming_level_window = 10``` to retrieve a 20 year window.<br><br>We can also modify the warming levels computed to include additional warming levels beyond the default. Let's select a few more by setting ```warming_level = [2.5, 3.0, 4.0]```. 

In [ ]:
get_data(
    variable = "Precipitation (total)", 
    downscaling_method = "Dynamical", 
    resolution = "45 km", 
    timescale = "monthly", 
    cached_area = "San Bernardino County", 
    approach = "Warming Level",
    
    # Modify warming level settings 
    warming_level_window = 10, 
    warming_level = [2.5, 3.0, 4.0]
)

### Example 3: Weather station data 
By default, the function retrieves non-bias-corrected gridded data, but you can also retrieve dynamically downscaled climate data that has been bias-corrected using historical weather station data. This data is retrieved at the unique grid cell(s) corresponding to the selected weather station(s). This data can be retrieved using the `data_type` and `stations` arguments. If you don't set the `stations` argument, the function will return all available weather stations-- a hefty data retrieval that takes a while to run and is therefore is not recommended. 

```
data_type = "Stations" # Return bias-corrected gridded data at a station(s) of interest 
data_type = "Gridded" # Return gridded data (function default) 
```

As of now, you can only retrieve hourly data for the variable "Air Temperature at 2m". You can also choose the resolution of the gridded data used in bias correction by setting the `resolution` argument to either "3 km" or "9 km".

#### Example 3a

In [ ]:
get_data(
    variable = "Air Temperature at 2m", # Required argument
    resolution = "9 km", # Required argument. Options: "9 km" or "3 km" 
    timescale = "hourly", # Required argument
    data_type = "Stations", # Required argument
    stations = "San Diego Lindbergh Field (KSAN)" # Optional argument. If no input, all weather stations are retrieved 
)

To see all the available weather station options, you can use the `get_subsetting_options()` function detailed at the top of this notebook. Simply set the `area_subset` function argument to `"Stations"`. 

In [ ]:
get_subsetting_options(area_subset="Stations") 

#### Example 3b
To demonstrate the flexibility of this function, let's make a few changes to the function argument in the code below: 
1) Retrieve more than one weather station. 
2) Change the resolution of the data used for bias-correction
3) Change the variable units (function default to degrees Kelvin, the native unit of the raw data)
4) Change the timescale to retrieve a 5 year period (function defaults to the entire historical record: 1980-2014)

Note that this function will take more time to run since we're retrieving more than one station. 

In [ ]:
get_data(
    variable = "Air Temperature at 2m", 
    resolution = "3 km",
    timescale = "hourly",
    data_type = "Stations",
    stations = [
        "San Francisco International Airport (KSFO)", 
        "Oakland Metro International Airport (KOAK)", 
    ],
    units = "degF", 
    time_slice = (2000,2005) 
)

#### Example 3c 
You can also retrieve data for the future period by setting the `scenario` argument to one or more SSPs. The code below will retrieve data for 2000-2035 for the Sacramento Executive Airport (KSAC) station under Shared Socioeconomic Pathway 3-7.0. Since we are retrieving data in both the historical and future period, we need to set the scenario to `['Historical Climate', 'SSP 3-7.0']`.

In [ ]:
get_data(
    variable = "Air Temperature at 2m", 
    resolution = "9 km",
    timescale = "hourly",
    data_type = "Stations",
    stations = "Sacramento Executive Airport (KSAC)",
    units = "degF", 
    time_slice = (2000,2035),
    scenario = ["Historical Climate", "SSP 3-7.0"]
)

## Exporting data

To save data as a file, call `export` and input your desired
1) data to export – an [xarray DataArray or Dataset](https://docs.xarray.dev/en/stable/user-guide/data-structures.html), as output by e.g. selections.retrieve()
2) output file name (without file extension)
3) file format ("NetCDF", "Zarr", or "CSV")

We recommend NetCDF or Zarr, which suits data and outputs from the Analytics Engine well – they efficiently store large data containing multiple variables and dimensions. Metadata will be retained in these files.

NetCDF or Zarr can be export locally (such as onto the JupyterHUB user partition). Optionally Zarr can be exported to an AWS S3 scratch bucket for storing very large exports.

CSV can also store Analytics Engine data with any number of variables and dimensions. It works the best for smaller data with fewer dimensions. The output file will be compressed to ensure efficient storage. Metadata will be preserved in a separate file.

CSV stores data in tabular format. Rows will be indexed by the index coordinate(s) of the DataArray or Dataset (e.g. scenario, simulation, time). Columns will be formed by the data variable(s) and non-index coordinate(s).

In [2]:
# First, load some data into an xarray object using get_data 
data_to_download = get_data(
    variable = "Air Temperature at 2m", 
    downscaling_method = "Dynamical", 
    resolution = "45 km", 
    timescale = "monthly", 
    scenario = "Historical Climate",
    time_slice = (2000,2001)
)

# Next, export the data to a netcdf file 
ck.export(data_to_download, filename="my_filename1", format="NetCDF") 

Exporting specified data to NetCDF...
Saving file locally as NetCDF4...


PermissionError: [Errno 13] Permission denied: '/workspaces/josh-external-sources/notebooks/my_filename1.nc'

Some additional file export formats are demonstrated below. The code has been commented out; simply remove the code comment hash (#) at beginning of the line to make the code executable. 

In [ ]:
# ck.export(data_to_use, filename="my_filename2", format="Zarr") # Zarr export locally

# ck.export(data_to_use, filename="my_filename3", format="Zarr", mode="s3") # Zarr export to S3

# ck.export(data_to_use, filename="my_filename4", format="CSV") # CSV export locally

Zarr format is technically a directory, not a single file, because it uses chunking to write and read data. This cloud-optimized format enables some unique benefits for performing data computations in a cloud computing environment like the AE Jupyter Hub, but can also make it tricky to delete. Because of that, we have built a simple helper function to facilitate easily deleting zarrs. 

In [ ]:
# ck.remove_zarr("my_filename2") # Helper function to delete Zarr directory tree

_______________

Attempt to download Riverside County data monthly data to NetCDF. 

In [2]:
precip_riverside = get_data(
    variable = "Precipitation (total)", 
    downscaling_method = "Statistical", 
    resolution = "3 km", 
    timescale = "monthly", 
    cached_area = "Riverside County", 
    approach = "Warming Level"
)

In [3]:
precip_riverside_annual = precip_riverside.sel(warming_level=2).sum(dim='time_delta').astype(float)
precip_riverside_annual

<xarray.DataArray np.str_('Precipitation (total)') (lat: 21, lon: 104,
                                                    simulation: 129)> Size: 2MB
dask.array<sum-aggregate, shape=(21, 104, 129), dtype=float64, chunksize=(21, 97, 1), chunktype=numpy.ndarray>
Coordinates:
    warming_level  float64 8B 2.0
  * lat            (lat) float32 84B 33.45 33.48 33.52 ... 34.02 34.05 34.08
  * lon            (lon) float32 416B -117.7 -117.6 -117.6 ... -114.5 -114.5
    spatial_ref    int64 8B 0
    centered_year  (simulation) int64 1kB 2037 2039 2040 2036 ... 2033 2039 2040
  * simulation     (simulation) <U48 25kB 'LOCA2_ACCESS-CM2_r1i1p1f1_historic...
Attributes:
    variable_id:           pr
    extended_description:  Total precipitation. Computed by summing total gri...
    units:                 kg m-2 s-1
    data_type:             Gridded
    resolution:            3 km
    frequency:             monthly
    location_subset:       ['Riverside County']
    approach:              Warming Level
    downscaling_method:    Statistical
    warming_level_window:  +/- 15 years from centered year
    institution:           UCSD

In [5]:
ck.export(precip_riverside_annual, filename="precip_riverside_annual", format="NetCDF") 

Exporting specified data to NetCDF...
Saving file locally as NetCDF4...


PermissionError: [Errno 13] Permission denied: '/workspaces/josh-external-sources/notebooks/precip_riverside_annual.nc'

________________


In [27]:
precip_riverside[precip_riverside['warming_level'] == 2]

<xarray.DataArray np.str_('Precipitation (total)') (warming_level: 1,
                                                    time_delta: 360, lat: 21,
                                                    lon: 104, simulation: 129)> Size: 811MB
dask.array<getitem, shape=(1, 360, 21, 104, 129), dtype=float64, chunksize=(1, 12, 21, 97, 1), chunktype=numpy.ndarray>
Coordinates:
  * warming_level  (warming_level) float64 8B 2.0
  * lat            (lat) float32 84B 33.45 33.48 33.52 ... 34.02 34.05 34.08
  * lon            (lon) float32 416B -117.7 -117.6 -117.6 ... -114.5 -114.5
    spatial_ref    int64 8B 0
  * time_delta     (time_delta) float64 3kB -180.0 -179.0 -178.0 ... 178.0 179.0
    centered_year  (simulation) int64 1kB 2037 2039 2040 2036 ... 2033 2039 2040
  * simulation     (simulation) <U48 25kB 'LOCA2_ACCESS-CM2_r1i1p1f1_historic...
Attributes:
    variable_id:           pr
    extended_description:  Total precipitation. Computed by summing total gri...
    units:                 kg m-2 s-1
    data_type:             Gridded
    resolution:            3 km
    frequency:             monthly
    location_subset:       ['Riverside County']
    approach:              Warming Level
    downscaling_method:    Statistical
    warming_level_window:  +/- 15 years from centered year
    institution:           UCSD

In [30]:
annual_precip = precip_riverside.groupby('centered_year').sum()

In [44]:
annual_sum = annual_precip.sel(warming_level=2).sum(dim='time_delta').astype(float)

In [47]:
for name, coord in annual_sum.coords.items():
    print(f"Coordinate: {name}, dtype: {coord.dtype}")

Coordinate: warming_level, dtype: float64
Coordinate: lat, dtype: float32
Coordinate: lon, dtype: float32
Coordinate: spatial_ref, dtype: int64
Coordinate: centered_year, dtype: int64


In [46]:
annual_sum

<xarray.DataArray np.str_('Precipitation (total)') (lat: 21, lon: 104,
                                                    centered_year: 45)> Size: 786kB
dask.array<sum-aggregate, shape=(21, 104, 45), dtype=float64, chunksize=(21, 97, 1), chunktype=numpy.ndarray>
Coordinates:
    warming_level  float64 8B 2.0
  * lat            (lat) float32 84B 33.45 33.48 33.52 ... 34.02 34.05 34.08
  * lon            (lon) float32 416B -117.7 -117.6 -117.6 ... -114.5 -114.5
    spatial_ref    int64 8B 0
  * centered_year  (centered_year) int64 360B 2017 2019 2020 ... 2065 2066 2067
Attributes: (12/19)
    variable_id:            pr
    extended_description:   Total precipitation. Computed by summing total gr...
    units:                  kg m-2 s-1
    data_type:              Gridded
    resolution:             3 km
    frequency:              monthly
    ...                     ...
    Analysis_package_name:  climakitae
    Version:                1.2.2
    Author:                 Cal-Adapt Analytics Engine Team
    Author_email:           analytics@cal-adapt.org
    Home_page:              https://github.com/cal-adapt/climakitae
    License:                BSD 3-Clause License

In [51]:
ck.export(annual_sum, filename="riverside_annual_sum", format="NetCDF") 

Exporting specified data to NetCDF...
Saving file locally with compression...


TypeError: No conversion path for dtype: dtype('<U4')